In [6]:
# criar o navegador
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico)

In [7]:
# caminho da página index (entrar no site da busca jurídica)
import os
caminho_jusserach = 'file://' + os.getcwd()
pasta_template = os.path.join(caminho_jusserach, 'paginas')
caminho_template = pasta_template + 'index_processo.html'
print(caminho_template) 

navegador.get(caminho_template)

file:///home/usuario/Documentos/Repositorys/NotaWeb/NotaWeb/JusSearch/paginasindex_processo.html


In [8]:
# importar a base de dados
import pandas as pd

caminho_arquivo = os.path.join(os.getcwd(), 'data')
tabela = pd.read_excel(caminho_arquivo + "/Processos.xlsx")
display(tabela)

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,NaN
1,João,Lawyer Alon,EB3792,Rio de Janeiro,NaN
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,NaN
3,Carol,Amanda,PC5197,São Paulo,NaN


In [9]:
from selenium.webdriver import ActionChains
from time import sleep

for linha in tabela.index:
    sleep(1)
    # para cada processo (linha da tabela)
    navegador.get(caminho_template)
    sleep(1)
    # abrir a lista de cidades
    botao = navegador.find_element(By.CLASS_NAME, 'dropdown-menu')
    ActionChains(navegador).move_to_element(botao).perform()
    
    cidade = tabela.loc[linha, "Cidade"]
    
    # selecionando o cidade
    navegador.find_element(By.PARTIAL_LINK_TEXT, cidade).click()
    
    # mudar para a nova aba
    aba_original = navegador.window_handles[0]
    indice = 1 + linha
    nova_aba = navegador.window_handles[indice]
    
    navegador.switch_to.window(nova_aba)
    
    # preencher o formulário com os dados de busca
    navegador.find_element(By.ID, 'nome').send_keys(tabela.loc[linha, "Nome"])
    navegador.find_element(By.ID, 'advogado').send_keys(tabela.loc[linha, "Advogado"])
    navegador.find_element(By.ID, 'numero').send_keys(tabela.loc[linha, "Processo"])

    # clicar em pesquisar
    navegador.find_element(By.CLASS_NAME, 'registerbtn').click()
    
    # confirmar a pesquisa
    alerta = navegador.switch_to.alert
    alerta.accept()
    
    # esperar o resultado da pesquisa e agir de acordo com o resultado
    while True:
        try:
            alerta = navegador.switch_to.alert
            break
        except:
            time.sleep(1)
    texto_alerta = alerta.text

    if "Processo encontrado com sucesso" in texto_alerta:
        alerta.accept()
        tabela.loc[linha, "Status"] = "Encontrado"
    else:
        tabela.loc[linha, "Status"] = "Não encontrado"
        alerta.accept()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".dropdown-menu"}
  (Session info: chrome=131.0.6778.85); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x563feb32d31a <unknown>
#1 0x563feae436e0 <unknown>
#2 0x563feae923e6 <unknown>
#3 0x563feae92681 <unknown>
#4 0x563feaed7b04 <unknown>
#5 0x563feaeb648d <unknown>
#6 0x563feaed4ed7 <unknown>
#7 0x563feaeb6203 <unknown>
#8 0x563feae84cc0 <unknown>
#9 0x563feae85c9e <unknown>
#10 0x563feb2fad0b <unknown>
#11 0x563feb2fec92 <unknown>
#12 0x563feb2e7b3c <unknown>
#13 0x563feb2ff807 <unknown>
#14 0x563feb2cd0df <unknown>
#15 0x563feb31c578 <unknown>
#16 0x563feb31c740 <unknown>
#17 0x563feb32c196 <unknown>
#18 0x7fbe0807dea7 start_thread


In [45]:
navegador.quit()
display(tabela)

tabela.to_excel("Processos Atualizado.xlsx")

,Nome,Advogado,Processo,Cidade,Status
0,Lira,Alon Lawyer,PC6592,Distrito Federal,Encontrado
1,João,Lawyer Alon,EB3792,Rio de Janeiro,Não encontrado
2,Amanda,Amanda mesmo,MM1043,Rio de Janeiro,Encontrado
3,Carol,Amanda,PC5197,São Paulo,Não encontrado
